In [2]:
import sys
import os
import cv2
import tensorflow.keras.backend as K
import numpy as np
from numpy import asarray
from core.networks_simple import UNET
from PIL import Image as im
K.set_image_data_format('channels_first')
os.environ['KERAS_BACKEND'] = "tensorflow"
vid_arr = np.zeros(shape=(2, 640, 352, 3), dtype="uint8")
image1 = im.open('./balloon/real/out-001.png')
image2 = im.open('./balloon/real/out-005.png')
vid_arr[0] = cv2.resize(asarray(image1), (352, 640))
vid_arr[1] = cv2.resize(asarray(image2), (352, 640))

model = UNET((6, 640, 352))
model.load_weights('./model_weights_for 512x256 5ep simple unet/model_weights.hdf5')

pred = model.predict(np.expand_dims(np.transpose(np.concatenate((vid_arr[0], vid_arr[1]), axis=2)/255., (2,0,1)), axis=0))
res = (np.transpose(pred[0], (1, 2, 0))*255).astype("uint8")
img = im.fromarray(res)
img.save('./balloon/real/out.png')
# img.resize((512, 512)).save('./now/water/out5.png')
# print(len(res))

In [35]:
import math
from skimage.metrics import structural_similarity as compare_ssim
def calculateMetrics(gt, result):
  psnr(gt, result)
  ssim = calculate_ssim(gt, result)
  print(f"SSIM: {ssim}%")

def psnr(img1, img2):
  mse = np.mean((img1 - img2) ** 2)
  if mse == 0:
    return 100
  # print(f"MSE: {mse}")
  PIXEL_MAX = 255.0
  psnr_val = 20 * math.log10(PIXEL_MAX / math.sqrt(mse))
  print(f"PSNR: {psnr_val}")

def calculate_ssim(gt, result):
  grayGT = cv2.cvtColor(gt, cv2.COLOR_BGR2GRAY)
  grayRes = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)
  (score, diff) = compare_ssim(grayGT, grayRes, full=True)
  diff = (diff * 255).astype("uint8")
  return score



In [25]:
gt = cv2.imread('./balloon/real/out-002.png')
pred = cv2.imread('./balloon/real/out.png')
calculateMetrics(gt, pred)

MSE: 30.665380859375
PSNR: 33.26431997939367
SSIM: 0.9109635637145302%


In [10]:
import sys
import os
import cv2
import tensorflow.keras.backend as K
import numpy as np
from numpy import asarray
from core.networks_simple import UNET
from PIL import Image as im
K.set_image_data_format('channels_first')
os.environ['KERAS_BACKEND'] = "tensorflow"


In [22]:

# print(len(res))
vid_arr = np.zeros(shape=(2, 640, 352, 3), dtype="uint8")
image1 = im.open('./balloon/real/out-001.png')
image2 = im.open('./balloon/real/out-003.png')
vid_arr[0] = cv2.resize(asarray(image1), (352, 640))
vid_arr[1] = cv2.resize(asarray(image2), (352, 640))


In [23]:
model = UNET((6, 640, 352))
model.load_weights('./model_weights_for 512x256 5ep simple unet/model_weights.hdf5')


In [24]:
  
pred = model.predict(np.expand_dims(np.transpose(np.concatenate((vid_arr[0], vid_arr[1]), axis=2)/255., (2,0,1)), axis=0))
res = (np.transpose(pred[0], (1, 2, 0))*255).astype("uint8")
img = im.fromarray(res)
img.save('./balloon/real/rec/out.png')


In [26]:
def recursive(frame1, frame2, num_recursions):
  if num_recursions == 0:
    yield frame1
  else:
    # Adds the batch dimension to all inputs before calling the interpolator,
    # and remove it afterwards.
    # time = np.full(shape=(1,), fill_value=0.5, dtype=np.float32)
    pred = model.predict(np.expand_dims(np.transpose(np.concatenate((frame1, frame2), axis=2)/255., (2,0,1)), axis=0))
    mid_frame = (np.transpose(pred[0], (1, 2, 0))*255).astype("uint8")
    yield from recursive(frame1, mid_frame, num_recursions - 1)
    yield from recursive(mid_frame, frame2, num_recursions - 1)


In [39]:
result = list(recursive(vid_arr[0], vid_arr[1], 2))
result.append(vid_arr[1])
len(result)

5

In [41]:
for i,img in enumerate(result):
  image = im.fromarray(img)
  image.save(f'./balloon/real/rec/{i}.png')


In [42]:
for i in range (5):
  gt = cv2.imread('./balloon/real/out-002.png')
  pre = cv2.imread(f'./balloon/real/rec/{i}.png')
  print(calculateMetrics(pre, gt))

PSNR: 35.2516582630667
SSIM: 0.9149156553871434%
None
PSNR: 32.24244130475458
SSIM: 0.8975399024036226%
None
PSNR: 33.58313431502054
SSIM: 0.9212649886683834%
None
PSNR: 32.27587569328877
SSIM: 0.8966286585768182%
None
PSNR: 34.97870251720799
SSIM: 0.9076569129998047%
None
